In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import joblib

import xgboost as xgb


from helpers import CleanData, KFoldCrossVal, feature_selection, feature_generation

## Preprocessing

In [ ]:
cleaner = CleanData()
cleaner.label_dict = joblib.load('intermediate/label_dict.pkl')

train = pd.read_csv('input/train.csv')
train = cleaner.clean(train, label_encode=True)

## Benchmark Model
10-time 5-fold cross-validation with Random Forest and no missing value handling

In [ ]:
gb_clf = xgb.XGBClassifier(early_stopping_rounds=5)
validator = KFoldCrossVal(n_repeats=1)
validator.fit(train, gb_clf)

validator.best_features.most_common()

In [ ]:
test = pd.read_csv('input/test.csv')
test = cleaner.clean(test, label_encode=True)

def make_submission(clf, df):
    pd.DataFrame({'LNR':test.index,
                  'RESPONSE': clf.predict_proba(df)[:,1]}).to_csv('submission.csv', index=False)
    
make_submission(validator.best_est, test)

## Impute missing

In [ ]:
# from sklearn.experimental import enable_iterative_imputer

# import joblib
# br_imputer = joblib.load('intermediate/br_imputer.pkl')
# imputed_list = joblib.load('intermediate/br_cols_imputed_list.pkl')

# train_imputed = train.copy()
# train_imputed[imputed_list] = br_imputer.transform(train_imputed[imputed_list])

# from sklearn.impute import IterativeImputer
# from sklearn.linear_model import BayesianRidge

# azdias = pd.read_csv('input/azdias.csv')
# azdias = cleaner.clean(azdias, label_encode=True)
# azdias_imputed = azdias.copy()
# del azdias
# azdias_imputed[imputed_list] = br_imputer.transform(azdias_imputed[imputed_list])

# br2 = BayesianRidge() #I forgot to impute `EINGEFUEGT_AM` in the first imputer
# imputer2 = IterativeImputer(br2) 

# azdias_imputed[imputed_list+['EINGEFUEGT_AM']] = imputer2.fit_transform(azdias_imputed[imputed_list+['EINGEFUEGT_AM']])

# def create_time_difference_variables(df):
#     time_cols = ['GEBURTSJAHR', 'EINGEZOGENAM_HH_JAHR','EINGEFUEGT_AM', 'MIN_GEBAEUDEJAHR']
#     for i in range(len(time_cols)-1):
#         for j in range(i+1, len(time_cols)):
#             df['diff_'+time_cols[i]+time_cols[j]] = df[time_cols[i]] - df[time_cols[j]]
#     return df
# azdias_imputed = create_time_difference_variables(azdias_imputed)
# azdias_imputed.isnull().sum().sum()

# train_imputed[imputed_list+['EINGEFUEGT_AM']] = imputer2.transform(train_imputed[imputed_list+['EINGEFUEGT_AM']])
# train_imputed = create_time_difference_variables(train_imputed)
# train_imputed.isnull().sum().sum()

# for c in azdias_imputed.columns:
#     # restrain the imputed values to be within the range of known values
#     if train[c].isnull().sum()==0:
#         continue
#     train_imputed[c] = np.clip(train_imputed[c], a_min=train[c].min(), a_max=train[c].max())
#     azdias_imputed[c] = np.clip(azdias_imputed[c], a_min=train[c].min(), a_max=train[c].max())

# azdias_imputed.to_csv('intermediate/azdias_imputed.csv', index=False)
# train_imputed.to_csv('intermediate/train_imputed.csv', index=False)

azdias_imputed = pd.read_csv('intermediate/azdias_imputed.csv')
train_imputed = pd.read_csv('intermediate/train_imputed.csv')

## Model experiment

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB

from sklearn.ensemble import RandomTreesEmbedding


### Run different models on features selected from Benchmark Model
Use the features that appear in the top 20 most important 20 out of 50 times

In [ ]:
selected_features = pd.read_csv('intermediate/top20_features_50_run.csv')
selected_features = list(selected_features.feature[selected_features.frequency>=5])

selected_features_union = list(set(selected_features + 
                                   ['D19_SOZIALES',
                                    'KBA05_MAXBJ',
                                    'D19_FREIZEIT',
                                    'KBA05_AUTOQUOT',
                                    'KBA05_ALTER2',
                                    'KBA13_VORB_3',
                                    'KBA05_KRSHERST3',
                                    'D19_BANKEN_ANZ_12',
                                    'AGER_TYP',
                                    'KBA13_BJ_2000',
                                    'D19_VERSAND_OFFLINE_DATUM',
                                    'SEMIO_KAEM',
                                    'diff_EINGEFUEGT_AMMIN_GEBAEUDEJAHR',]))

In [ ]:
validator = KFoldCrossVal()

log_clf = LogisticRegression(class_weight='balanced', random_state=7, solver='lbfgs')
nb_clf = BernoulliNB()
rf_clf = xgb.XGBRFClassifier(scale_pos_weight=80, n_jobs=-1, n_estimators=50, )
gb_clf = xgb.XGBClassifier(scale_pos_weight=80, n_jobs=-1, early_stopping_rounds=10, )


In [ ]:
for clf in [rf_clf, gb_clf]:
    print(clf.__module__)
    validator.fit(train[selected_features+['RESPONSE']], clf, get_best_features=False)
    print('std', np.std(validator.test_evals))

In [ ]:
for clf in [log_clf, nb_clf]:
    print(clf.__module__)
    selected_features_cats = [v for v in selected_features if v in cleaner.categorical_cols]
    train_imputed_dummied = pd.get_dummies(train_imputed[selected_features+['RESPONSE']], drop_first=True, 
                                           columns=selected_features_cats)
    validator.fit(train_imputed_dummied, clf, get_best_features=False)
    print('std', np.std(validator.test_evals))

### Run models on PCA features
Performs really bad :(

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

estimators = [('scaler', MinMaxScaler()),
              ('linear_pca', PCA(random_state=7, n_components=248))]
print('Making pipeline...')
process_pipe = Pipeline(estimators)
azdias_dummied = pd.get_dummies(azdias, columns=cleaner.categorical_cols, drop_first=True)
print('Fitting pipeline...')
process_pipe.fit(azdias_dummied)


In [ ]:
# train_imputed_dummied = pd.get_dummies(train_imputed, columns=cleaner.categorical_cols, drop_first=True)
train_imputed_dummied['TITEL_KZ_2.0'] = 0
train_imputed_dummied_pca = pd.DataFrame(process_pipe
                                         .transform(train_imputed_dummied[azdias_dummied.columns]),
                                         index=train_imputed_dummied.index
                                        )
train_imputed_dummied_pca['RESPONSE'] = train_imputed_dummied.RESPONSE

train_imputed_dummied_pca['RESPONSE'].isnull().sum()

### Try Tree-embedding features

#### Tree embedding features trained on `train` dataset

In [ ]:
rf_clf

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import BernoulliNB


rf_clf = xgb.XGBRFClassifier(max_depth=3, min_child_weight=30, n_estimators=50,
                n_jobs=-1, scale_pos_weight=80)
grd_enc = OneHotEncoder(categories='auto', drop='first')
lm = LogisticRegression(solver='lbfgs', max_iter=1000)
nb = BernoulliNB()

skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=7)

X = train.drop(columns=['RESPONSE'])
y = train.RESPONSE


def train_tree_embed(X, y, clf, grd_enc, lm, nb, fitted=False):
    log_test_results = []
    nb_test_results = []
    i=0
    best_auc = 0
    train_auc = None
    for train_index, test_index in skf.split(X, y):
        print(i)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if not fitted:
            clf.fit(X_train, y_train)
        grd_enc.fit(clf.apply(pd.concat((X_train, X_test))))
        X_train_enc = grd_enc.transform(clf.apply(X_train))
        lm.fit(X_train_enc, y_train)
        nb.fit(X_train_enc, y_train)

        X_test_enc = grd_enc.transform(clf.apply(X_test))
        y_pred_lm = lm.predict_proba(X_test_enc)[:, 1]
        y_pred_nb = nb.predict_proba(X_test_enc)[:, 1]

        res_log = roc_auc_score(y_test, y_pred_lm)
        res_nb = roc_auc_score(y_test, y_pred_nb)
        if res_log > best_auc:
            best_auc = res_log
            train_auc = roc_auc_score(y_train, lm.predict_proba(X_train_enc)[:, 1])
            best_est = lm
        if res_nb > best_auc:
            best_auc = res_nb
            train_auc = roc_auc_score(y_train, nb.predict_proba(X_train_enc)[:, 1])
            best_est = nb
        print('Log:',res_log, '   Nb:', res_nb)
        log_test_results.append(res_log)
        nb_test_results.append(res_nb)
        i+=1
    print('Log:', np.mean(log_test_results), '        std:', np.std(log_test_results))
    print('NB:', np.mean(nb_test_results), '        std:', np.std(nb_test_results))
    print(best_est, best_auc, train_auc)
    return log_test_results, nb_test_results, clf, grd_enc, best_est, best_auc, train_auc

log_test_results, nb_test_results, rf_clf, grd_enc, best_est, best_auc, train_auc = train_tree_embed(X, y, rf_clf, grd_enc, lm, nb)

In [ ]:
X = train[selected_features]
y = train.RESPONSE

log_test_results, nb_test_results, rf_clf, grd_enc, best_est, best_auc, train_auc = train_tree_embed(X, y, rf_clf, grd_enc, lm, nb)
print(np.mean(log_test_results))
print(np.mean(nb_test_results))

In [ ]:
X = train[selected_features]
y = train.RESPONSE

_, _, _, _, best_est, best_auc, train_auc = train_tree_embed(X, y, gb_clf, grd_enc, lm, nb)

In [ ]:
X = train.drop(columns=['RESPONSE'])
y = train.RESPONSE

_, _, _, _, best_est, best_auc, train_auc = train_tree_embed(X, y, gb_clf, grd_enc, lm, nb)

### Tree-embedding features trained on `azdias` and `customers`
- Concatenate `azdias` and `customers`
- Create new variable `RESPONSE`: 1 for `customers` and 0 for `azdias`

In [ ]:
azdias = pd.read_csv('input/azdias.csv')
customers = pd.read_csv('input/customers.csv')

customers = customers[azdias.columns]

azdias = cleaner.clean(azdias, label_encode=True)
customers = cleaner.clean(customers, label_encode=True)

azdias['RESPONSE'] = 0
customers['RESPONSE'] = 1

full = pd.concat((azdias, customers))
del azdias
del customers
from sklearn.model_selection import train_test_split
gb_clf_full = xgb.XGBClassifier(scale_pos_weight=2, n_jobs=-1, n_estimators=30, early_stopping_rounds=10)

full = full.sample(frac=1, random_state=7)

X = full.drop(columns=['RESPONSE'])#[selected_features_union]#
y = full.RESPONSE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

gb_clf_full.fit(X_train, y_train, early_stopping_rounds=10,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                eval_metric='auc',
                verbose=True
                )


In [ ]:
grd_enc = OneHotEncoder(categories='auto', drop='first')
grd_lm = LogisticRegression(solver='lbfgs', max_iter=1000)
nb = BernoulliNB()

skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

X = train.drop(columns=['RESPONSE'])
y = train.RESPONSE

log_test_results, nb_test_results, gb_clf_full, grd_enc, best_est, best_auc, train_auc = train_tree_embed(X, y, gb_clf_full, grd_enc, lm, nb, fitted=True)

print(np.mean(log_test_results))
print(np.mean(nb_test_results))
print(best_est, best_auc, train_auc)

In [ ]:
gb_clf_full_reduced = xgb.XGBClassifier(scale_pos_weight=2, n_jobs=-1, n_estimators=30, early_stopping_rounds=10)

full = full.sample(frac=1, random_state=7)

X = full[selected_features_union]
y = full.RESPONSE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

gb_clf_full_reduced.fit(X_train, y_train, early_stopping_rounds=5,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                eval_metric='auc',
                verbose=True
                )

X = train[selected_features_union]
y = train.RESPONSE

log_test_results, nb_test_results, gb_clf_full, grd_enc, best_est, best_auc, train_auc = train_tree_embed(X, y, 
                                                                                                          gb_clf_full_reduced, grd_enc, lm, nb, fitted=True)

In [ ]:
rf_clf_full_reduced = xgb.XGBRFClassifier(scale_pos_weight=1, n_jobs=-1, n_estimators=50)

X = full.drop(columns=['RESPONSE'])
y = full.RESPONSE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_clf_full_reduced.fit(X_train, y_train, early_stopping_rounds=5,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                eval_metric='auc',
                verbose=True
                )

X = train.drop(columns=['RESPONSE'])
y = train.RESPONSE

log_test_results, nb_test_results, rf_clf_full_reduced, grd_enc, best_est, best_auc, train_auc = train_tree_embed(X, y, 
                                                                                                          rf_clf_full_reduced, grd_enc, lm, nb, fitted=True)

In [ ]:
X = train[selected_features_union]
y = train.RESPONSE

log_test_results = []
log_train_results = []
nb_test_results = []
nb_train_results = []

clf = gb_clf_full_reduced
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    grd_enc.fit(clf.apply(X_train))
    X_train_enc = grd_enc.fit_transform(clf.apply(X_train))
    lm.fit(X_train_enc, y_train)
    nb.fit(X_train_enc, y_train)

    X_test_enc = grd_enc.transform(clf.apply(X_test))

    res_log = roc_auc_score(y_test, lm.predict_proba(X_test_enc)[:, 1])
    train_auc_log = roc_auc_score(y_train, lm.predict_proba(X_train_enc)[:, 1])
    performance_log = res_log**2*train_auc_log
    
    res_nb = roc_auc_score(y_test, nb.predict_proba(X_test_enc)[:, 1])
    train_auc_nb = roc_auc_score(y_train, nb.predict_proba(X_train_enc)[:, 1])
    performance_nb = res_nb**2*train_auc_nb
        
    log_test_results.append(res_log)
    log_train_results.append(train_auc_log)
    nb_test_results.append(res_nb)
    nb_train_results.append(train_auc_nb)
    
log_results = pd.DataFrame({'log_test_results': log_test_results, 'log_train_results': log_train_results})
nb_results = pd.DataFrame({'nb_test_results': nb_test_results, 'nb_train_results': nb_train_results})

print('Log:', np.mean(log_test_results), '        std:', np.std(log_test_results))
print('NB:', np.mean(nb_test_results), '        std:', np.std(nb_test_results))


### Transfer learning

In [ ]:
roc_auc_score(train.RESPONSE, rf_clf_full_reduced.predict_proba(train.drop(columns=['RESPONSE']))[:,1])

### Negative sampling
Find most similar negative instances to each positive instance

In [ ]:
import pandas as pd
from scipy.spatial import KDTree

train_yes = train[train.RESPONSE==1][selected_features].fillna(-3000)
train_no = train[train.RESPONSE==0][selected_features].fillna(-3000)

import sys
sys.setrecursionlimit(10000)

kdB = KDTree(train_no.values, leafsize=20)
print(kdB.query(train_yes.values[:1], k=3)[-1])

In [ ]:
train_no_match = list(set(kdB.query(train_yes.values, k=5)[-1].reshape(-1)))
train_no_match = train.iloc[train_no_match]
train_no_match['RESPONSE'] = 0

train_yes = train[train.RESPONSE==1]
train_yes['RESPONSE'] = 1

train_reduced = pd.concat((train_no_match, train_yes)).sample(frac=1)

In [ ]:
rf_clf_reduced = xgb.XGBRFClassifier(random_state=7, n_jobs=-1, scale_pos_weight=5)
validator.fit(train_reduced, rf_clf_reduced)

In [ ]:
selected_features2 = [u for u,v in validator.best_features.most_common() if v>=5]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
log_pip = Pipeline([('scale', MinMaxScaler()), ('log', LogisticRegression())])
selected_features_union_reduced = list(set(selected_features)|set(selected_features2))
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(train_reduced[selected_features_union_reduced], 
                                                        train_reduced.RESPONSE, stratify= train_reduced.RESPONSE,
                                                        test_size=0.2, random_state=i,
                                                       )
    log_pip.fit(X_train, y_train)
    print(roc_auc_score(y_test, log_pip.predict_proba(X_test)[:,1]))

## Tune rf

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_test1 = {
    'max_depth':np.arange(3,7),#4
    'min_child_weight':np.arange(10, 100, 10), #25
    'n_estimators':np.arange(10, 100, 10), #50
    'scale_pos_weight':[1, 5, 10, 20, 30, 40, 60,80, 100],
    'reg_alpha': [0.001, 0.01, 0.1, 1],
    'learning_rate': [1, 0.3, 0.1, 0.01, .005],
    'colsample_bytree': [0.2, 0.5, 0.8, 1],
    'subsample': np.arange(7, 11, 1)/10,
    'gamma': [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2]
}

gsearch1 = RandomizedSearchCV(estimator = xgb.XGBRFClassifier(learning_rate=0.1, 
                                                    n_estimators=50, max_depth=3,
                                                    gamma=0, subsample=1, 
                                                    objective= 'binary:logistic', 
                                                    n_jobs=-1, scale_pos_weight=1, 
                                                    ),
                              param_distributions=param_test1,
                              scoring='roc_auc',
                              n_iter=200,
                              cv=5, 
                              n_jobs=-1,
                              random_state=7,
                              return_train_score=True
                             )

from time import time
start = time()
gsearch1.fit(train.drop(columns=['RESPONSE']),train['RESPONSE'])
print("RandomizedSearchCV took %.2f seconds" % (time() - start))


In [ ]:
gsearch1_res = pd.DataFrame(gsearch1.cv_results_)
gsearch1_res.groupby('param_max_depth').mean_test_score.agg(['mean', 'count'])

In [ ]:
gsearch1_res.groupby('param_scale_pos_weight').mean_test_score.agg(['mean', 'count'])

In [ ]:
gsearch1.best_params_

In [83]:
gsearch2 = RandomizedSearchCV(estimator = xgb.XGBRFClassifier(learning_rate =0.1, 
                                                    n_estimators=50, max_depth=3,
                                                    gamma=0, subsample=1, 
                                                    objective= 'binary:logistic', 
                                                    n_jobs=-1, scale_pos_weight=1, 
                                                    ),
                              param_distributions=param_test1,
                              scoring='roc_auc',
                              n_iter=200,
                              cv=5, 
                              n_jobs=-1,
                              random_state=7,
                              return_train_score=True
                             )


start = time()
gsearch2.fit(train[selected_features_union_reduced],train['RESPONSE'])
print("RandomizedSearchCV took %.2f seconds" % (time() - start))

RandomizedSearchCV took 1019.26 seconds


In [86]:
param_results = pd.DataFrame(gsearch2.cv_results_)
for col in gsearch2.cv_results_:
    if col.startswith('param_'):
        print(param_results.groupby(col)['mean_test_score'].agg(['mean', 'count']))

                     mean  count
param_subsample                 
0.7              0.746499     52
0.8              0.759631     53
0.9              0.754869     39
1.0              0.749529     56
                            mean  count
param_scale_pos_weight                 
1                       0.669326     23
5                       0.765430     29
10                      0.766513     20
20                      0.764664     15
30                      0.763614     27
40                      0.763712     18
60                      0.761698     27
80                      0.763066     18
100                     0.758013     23
                     mean  count
param_reg_alpha                 
0.001            0.761937     50
0.010            0.762446     51
0.100            0.759358     46
1.000            0.727922     53
                        mean  count
param_n_estimators                 
20                  0.756748     21
30                  0.742690     26
40                  

In [84]:
print(gsearch2.best_params_)

print(gsearch2.best_score_)

{'subsample': 0.8, 'scale_pos_weight': 10, 'reg_alpha': 1, 'n_estimators': 20, 'min_child_weight': 70, 'max_depth': 5, 'learning_rate': 0.005, 'gamma': 0.1, 'colsample_bytree': 0.8}
0.7738409200929489


In [91]:
param_test2 = {
    'min_child_weight':np.arange(50, 150, 10),
    'n_estimators':np.arange(10, 100, 10),
    'reg_alpha': [0.001, 0.003, 0.01, 0.1, 1, 1.3, 1.5],
    'learning_rate': [0.01, .005, 0.001, 0.0005],
    'colsample_bytree': np.arange(2, 11, 1)/10,
    'subsample': np.arange(7, 11, 1)/10,
    'gamma': [0.01, 0.1, 0.3, 0.5]
}
gsearch3 = RandomizedSearchCV(estimator = xgb.XGBRFClassifier(learning_rate =0.1, 
                                                    n_estimators=30, max_depth=5,
                                                    subsample=1, 
                                                    min_child_weight= 40,          
                                                    objective= 'binary:logistic', 
                                                    n_jobs=-1, scale_pos_weight=10,          
                                                    ),
                              param_distributions=param_test2,
                              scoring='roc_auc',
                              n_iter=100,
                              cv=5, 
                              n_jobs=-1,
                              random_state=7,
                              return_train_score=True
                             )


start = time()
gsearch3.fit(train[selected_features_union_reduced],train['RESPONSE'])
print("RandomizedSearchCV took %.2f seconds" % (time() - start))

RandomizedSearchCV took 476.23 seconds


In [96]:
print(gsearch3.best_params_)

print(gsearch3.cv_results_['std_test_score'][88])

{'subsample': 0.8, 'reg_alpha': 1.3, 'n_estimators': 20, 'min_child_weight': 100, 'learning_rate': 0.005, 'gamma': 0.3, 'colsample_bytree': 0.7}
0.025005587440342206


In [93]:
make_submission(gsearch3.best_estimator_, test[selected_features_union_reduced])

In [102]:
!zip capstone.zip report.pdf report.ipynb README.md proposal.pdf helpers.py intermediate/*

updating: report.pdf (deflated 22%)
updating: report.ipynb (deflated 74%)
updating: README.md (deflated 48%)
updating: proposal.pdf (deflated 2%)
updating: helpers.py (deflated 70%)
  adding: intermediate/br_cols_imputed_list.pkl (deflated 59%)
  adding: intermediate/br_imputer.pkl (deflated 32%)
  adding: intermediate/features.pkl (deflated 49%)
  adding: intermediate/label_dict.pkl (deflated 75%)
  adding: intermediate/top20_features_50_run.csv (deflated 57%)
  adding: intermediate/transfer_learning.pkl (deflated 41%)
